In [91]:
#importing libraries
import numpy as np 
import pandas as pan 
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
rawData = pan.read_table("data/moviereviews.tsv")

In [6]:
rawData.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [7]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2000 non-null   object
 1   review  1965 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


**Checking what the data looks like at this point**

In [8]:
rawData["review"][0]

'how do films like mouse hunt get into theatres ? \nisn\'t there a law or something ? \nthis diabolical load of claptrap from steven speilberg\'s dreamworks studio is hollywood family fare at its deadly worst . \nmouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . \nwriter adam rifkin and director gore verbinski are the names chiefly responsible for this swill . \nthe plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . \ndeciding to check out the long-abandoned house , they soon learn that it\'s worth a fortune and set about selling it in auction to the highest bidder . \nbut battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . \nthe story alter

**The control characters are actually part of the formatting, and thus can teach some things about the user of the review**
<br/>
**They will not be removed**<br/>
**The value can be read in its human readable form if printed**


In [9]:
print(rawData["review"][0])

how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates betwee

In [16]:
rawData["label"][1]

'neg'

**Adding a col that turns the label into an integer to be able to treat it**

In [29]:
rawData["positive"] = rawData.apply(lambda row: 1 if row.label =="pos" else 0, axis=1)

In [34]:
rawData.head()

,label,review,positive
0,neg,how do films like mouse hunt get into theatres...,0
1,neg,some talented actresses are blessed with a dem...,0
2,pos,this has been an extraordinary year for austra...,1
3,pos,according to hollywood movies made in last few...,1
4,neg,my first press screening of 1998 and already i...,0


In [58]:
rawData = rawData[pan.notna(rawData['review'])]
rawData = rawData[pan.notna(rawData['positive'])]

**Training model**

In [59]:
from sklearn.model_selection import train_test_split

In [85]:
train, test_ds = train_test_split(rawData, test_size=0.3, random_state=413)

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [82]:
train

,label,review,positive
1041,pos,sometimes you just have to tip your hat to a f...,1
682,pos,note : some may consider portions of the follo...,1
62,neg,senseless ( r ) marlon wayans is a very talent...,0
626,pos,waiting at the train station near the beginnin...,1
336,neg,perhaps if the impostors didn't have so much g...,0
...,...,...,...
1824,pos,"an energetic , visually stunning , but intelle...",1
1112,pos,"synopsis : in this cultural exploration , a ch...",1
1967,neg,"whether you like the beatles or not , nobody w...",0
237,pos,"the computer-animated comedy "" shrek "" is desi...",1


In [63]:
# Create a pipeline which transforms phrases into normalized feature vectors and uses a bayes estimator
model_name = 'bayes classifier'

pipeline_bayes = Pipeline([
                ('count', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('gnb', MultinomialNB()),
                ])

# Train model using the created sklearn pipeline
model_bayes = pipeline_bayes.fit(train['review'], train['positive'])

**The model seems rather fited**

In [88]:
model_bayes.score(test['review'], test['positive'])

0.7576271186440678

**Testing the model**

In [66]:
testphrases = ['This is the worst movie I ever had the displeasurre to see', 'I FUCKING love this movie', 'That movie is kinda mid tbh', 'Completetly and utterly bad']
for testphrase in testphrases:
    resultx = model_bayes.predict([testphrase]) # use model_bayes for predictions with the other model
    dict = {0: 'Negative',  1: 'Positive'}
    print(testphrase + '-> ' + dict[resultx[0]])

This is the worst movie I ever had the displeasurre to see-> Negative
I FUCKING love this movie-> Positive
That movie is kinda mid tbh-> Negative
Completetly and utterly bad-> Negative


**Saving the model in a file**

In [69]:
from joblib import dump
dump(model_bayes, 'filename.joblib') 

['filename.joblib']

In [71]:
!pip install gradio
from joblib import load

  Using cached gradio-3.16.1-py3-none-any.whl (14.2 MB)


ERROR: Could not install packages due to an OSError: [WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'c:\\python39\\Lib\\site-packages\\altair\\vegalite\\v4\\tests\\test_api.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 101.4/101.4 KB 1.2 MB/s eta 0:00:00
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
  Using cached orjson-3.8.5-cp39-none-win_amd64.whl (202 kB)
  Using cached uvicorn-0.20.0-py3-none-any.whl (56 kB)
     -------------------------------------- 323.5/323.5 KB 1.7 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached markdown_it_py-2.1.0-py3-none-any.whl (84 kB)
  Using cached fastapi-0.89.1-py3-none-any.whl (55 kB)
  Using cached pycryptodome-3.16.0-cp35-abi3-win_amd64.whl (1.7 MB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
  Using cached ffmpy-0.3.0.tar.gz (4.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached python-multipart-0.0.5.tar.gz (32 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached too

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 14.2/14.2 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 KB 2.4 MB/s eta 0:00:00
     -------------------------------------- 812.8/812.8 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 55.8/55.8 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 71.5/71.5 KB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 56.9/56.9 KB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 202.7/202.7 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached websockets-10.4-cp39-cp39-win_amd64.whl (101 kB)
  Using cached aiohttp-3.8.3-cp39-cp39-win_amd64.whl (323 kB)
     ---------------------------

In [73]:
import gradio as gr

# Load the model
model_bayes = load('filename.joblib')

# Prediction function
def make_prediction(user_sentence):
  
  prediction = model_bayes.predict([user_sentence])
  dict = {0: 'Negative', 1: 'Positive'}
  return dict[prediction[0]]

title = "Sentiment Analysis movie Reviews"
description = "<p style='text-align: center'>Identifier si un commentaire sur un film est negatif ou positif<br/>Entrainer sur .</p>"
examples = ["The movie was...bad. The actors suck, the camerawork is shabby at best and the scenario is completly non-sensical", "This is amazing !"]

app = gr.Interface(fn=make_prediction, title=title, description=description, examples=examples, inputs=gr.TextArea(), outputs=gr.Label(num_top_classes=3))

app.launch(share=True)


Running on local URL:  http://127.0.0.1:7861

Could not create share link, please check your internet connection.


Limits:

First, this method has been trained on specificly english data with a rather professional way of writting
So, it isn't fit to use on more comptemporary or 